# Survival Analysis :  Quick Implementation

In [1]:
#install import stuff, you need to login and give google permissions twice
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!pip install pandas==0.22
!pip install lifelines

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
W: chmod 0700 of directory /var/lib/apt/lists/partial failed - SetupAPTPartialDirectory (1: Operation not permitted)
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#mount google drive
%cd
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
#navigating your google drive and saving files inside.
%cd
%ls
%cd drive/
%mkdir github_fromXA
%cd github_fromXA
%rm -r Survival-Analysis-Lifelines
!git clone https://github.com/leexa90/Survival-Analysis-Lifelines.git
#download model parameters
%ls -lh */
%ls -lh
%cd Survival-Analysis-Lifelines
%ls -lh

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
pd.set_option('display.max_columns',60)
pd.set_option('display.max_rows', 60)
%pylab inline
figsize(12,6)

### We will be using Telco Customer Churn data from IBM Watson Analytics
https://www.ibm.com/communities/analytics/watson-analytics-blog/Telco-Customer-Churn/

In [ ]:
##  create a dataframe
df = pd.read_excel("./WA_Fn-UseC_-Telco-Customer-Churn.xlsx") 

In [ ]:
'''data description

Customers who left within the last month – the column is called Churn

Services that each customer has signed up for – phone, multiple lines, internet, 
online security, online backup, device protection, tech support, and streaming TV and movies

Customer account information – how long they’ve been a customer, contract, payment method, 
paperless billing, monthly charges, and total charges

Demographic info about customers – gender, age range, and if they have partners and dependents

'''

In [2]:
## Have a first look at the data
df.head()

NameError: name 'df' is not defined

In [ ]:
#get all columns
df.keys()

In [ ]:
#view churn column , row 2 - 17 #python index starts from 0
df['Churn'].iloc[1:17]

In [ ]:
#percentage of churn customers
np.mean(df['Churn']=='Yes')

In [ ]:
# unique entries of tech support
pd.unique(df['TechSupport'])

In [ ]:
# access churn column from those who are on Tech support
df[df['TechSupport']=='Yes']['Churn']

In [ ]:
## Convert TotalCharges to numeric
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')

## Replace yes and No in the Churn column to 1 and 0. 1 for the event and 0 for the censured data.
df['Churn']=(df['Churn']=='Yes')*1

In [ ]:

for i in pd.unique(df['Contract']):
    print ('type of contract:',i)
    plt.hist(df[df['Contract']==i]['Churn'],density=True,bins=(0,1,2))
    plt.xticks([0.5,1.5],labels=[0,1])
    plt.xlabel('churn ==1, no churn ==0')
    plt.ylabel('normazlied prob density')
    plt.title(i)
    plt.show()

In [ ]:
#checking column for missing entries
df.info()

In [ ]:
#customers with missing values, fill in with median
df[df['TotalCharges'].isnull()]

In [ ]:
## Impute the null value with the median value
df.TotalCharges.fillna(value=df['TotalCharges'].median(),inplace=True)

In [ ]:
## Create a list of Categorical Columns
cat_cols= [i  for i in df.columns if df[i].dtype==object]
cat_cols.remove('customerID')  ## customerID has been removed because it is unique for all the rows.

In [ ]:
## lets have a look at the categories and their distribution in all the categorical columns.

for i in cat_cols:
    print('Column Name: ',i)
    print(df[i].value_counts())
    print('-----------------------------')

In [ ]:
print ('all variables:',df.keys())
variables = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges']

In [ ]:
df_processed = pd.get_dummies(df[variables])

In [ ]:
from sklearn import tree
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
X = pd.get_dummies(df_processed,drop_first=False)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(     
                            X, y, test_size=0.2, random_state=2)


clf = tree.DecisionTreeClassifier(max_depth=2,criterion='entropy',min_samples_leaf=25)
clf = clf.fit(X_train, y_train)
y_test_pred = clf.predict_proba(X_test)[:,1]
y_train_pred = clf.predict_proba(X_train)[:,1]
print ('train,test roc:',metrics.roc_auc_score(y_train,y_train_pred),metrics.roc_auc_score(y_test,y_test_pred))
fpr, tpr, thresholds  = metrics.roc_curve(y_train,y_train_pred)
plt.plot([0,1],[0,1],'r--',label='Chance')
plt.plot(fpr,tpr,'b',label='Train results')
fpr, tpr, thresholds  = metrics.roc_curve(y_test,y_test_pred)
plt.plot(fpr,tpr,'g',label='Test results')

plt.xlabel('True positive rate')
plt.ylabel('False positive rate')
plt.legend()
plt.show()

In [ ]:
import graphviz 
import pydot

dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=X.keys(),  
                      class_names="Survived",  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.render("Churn",format='png') 
graph

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(20,20))
plt.imshow(plt.imread('./Explain.png'))
plt.show()

In [ ]:
#making a more complicated model
clf = tree.DecisionTreeClassifier(max_depth=4,criterion='entropy',min_samples_leaf=25)
clf = clf.fit(X_train, y_train)
y_test_pred = clf.predict_proba(X_test)[:,1]
y_train_pred = clf.predict_proba(X_train)[:,1]
print ('train,test roc:',metrics.roc_auc_score(y_train,y_train_pred),metrics.roc_auc_score(y_test,y_test_pred))
fpr, tpr, thresholds  = metrics.roc_curve(y_train,y_train_pred)
plt.plot([0,1],[0,1],'r--',label='Chance')
plt.plot(fpr,tpr,'b',label='Train results')
fpr, tpr, thresholds  = metrics.roc_curve(y_test,y_test_pred)
plt.plot(fpr,tpr,'g',label='Test results')

plt.xlabel('True positive rate')
plt.ylabel('False positive rate')
plt.legend()
plt.show()

In [ ]:
import graphviz 
import pydot

dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=X.keys(),  
                      class_names="Survived",  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.render("Churn",format='png') 
graph

# Stradegies to reduce churn